In [4]:
# 动态创建类

def creat_class(name):
    if name == "user":
        class User:
            def __str__(self):
                return "user"
        return User
    elif name == "company":
        class Company:
            def __str__(self):
                return "Company"
        return Company

In [7]:
# 创建一个类

MyClass = creat_class("user")
my_obj = MyClass()
print(type(my_obj))

<class '__main__.creat_class.<locals>.User'>


使用type方法创建一个类

> type(class_name, bases, dict) -> a new type
type函数接收3个参数

1: 要被创建的类名   --str

2: 被创建的类的父类  --tuple

3: 该类中的方法和属性  -- dict

In [16]:
class BaseClass:
    def answer(self):
        return "I am BaseClass"


def say(self):
    return "I am user"

User = type("User", (BaseClass, ), {"name": "user", "say":say})

my_obj = User()

print(my_obj.name)
print(my_obj.say())
print(my_obj.answer())

user
I am user
I am BaseClass


In [24]:
# 元类
class MetaClass(type):
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls, *args, **kwargs)
        

class User(metaclass=MetaClass):
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

my_obj = User("User")
print(my_obj)

User


通过元类实现orm

In [ ]:
class User:
    name = 